In [117]:
import torch
import torch.nn as nn
import  torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [34]:
data = pd.read_csv("train.csv")

X = data.iloc[:, 3:]
y = data.iloc[:, 1:3]
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.80)

In [94]:
EPOCHS = 100
BATCH_SIZE = 64
LEARNING_RATE = 0.001
NUM_FEATURES = len(X.columns)

In [95]:
class RegressionDataset(Dataset):

    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)

train_dataset = RegressionDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
test_dataset = RegressionDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float())
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [108]:
# Defining the model
class ANN(nn.Module):
    def __init__(self, num_features):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(num_features, 10)
        self.fc2 = nn.Linear(10, 12)
        self.fc3 = nn.Linear(12, 20)
        self.fc4 = nn.Linear(20, 12)
        self.fc5 = nn.Linear(12, 10)
        self.fc6 = nn.Linear(10, 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, inputs):
        x = self.relu(self.dropout(self.fc1(inputs)))
        x = self.relu(self.dropout(self.fc2(x)))
        x = self.relu(self.dropout(self.fc3(x)))
        x = self.relu(self.dropout(self.fc4(x)))
        x = self.relu(self.dropout(self.fc5(x)))
        x = self.fc6(x)
        return x

    def predict(self, test_inputs):
        x = self.relu(self.fc1(test_inputs))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.relu(self.fc5(x))
        x = self.fc6(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [109]:
model = ANN(NUM_FEATURES).to(device)
print(model)

ANN(
  (fc1): Linear(in_features=6, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=12, bias=True)
  (fc3): Linear(in_features=12, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=12, bias=True)
  (fc5): Linear(in_features=12, out_features=10, bias=True)
  (fc6): Linear(in_features=10, out_features=2, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
)


In [110]:
model = ANN(NUM_FEATURES).to(device)
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)
loss_stats = {"train" : [], "test" : []}

In [111]:
print("Begin Training!!")

for epoch in range(1, EPOCHS):
    train_epoch_loss = 0
    for idx, (X_train_batch, y_train_batch) in enumerate(train_dataloader):
        X_train_batch = X_train_batch.to(device)
        y_train_batch = y_train_batch.to(device)
        optimizer.zero_grad()
        y_train_preds = model(X_train_batch)
        train_loss = criterion(y_train_preds, y_train_batch.unsqueeze(1))
        train_loss.backward()
        optimizer.step()
        train_epoch_loss += train_loss.item()
        
    loss_stats["train"].append(train_epoch_loss)
    print(f'Epoch {epoch}: | Train Loss: {train_epoch_loss/len(train_dataloader):.5f}')

Begin Training!!
Epoch 1: | Train Loss: 14.54848
Epoch 2: | Train Loss: 11.42260
Epoch 3: | Train Loss: 11.19509
Epoch 4: | Train Loss: 11.05411
Epoch 5: | Train Loss: 10.90995
Epoch 6: | Train Loss: 10.83637
Epoch 7: | Train Loss: 10.76522
Epoch 8: | Train Loss: 10.72551
Epoch 9: | Train Loss: 10.68430
Epoch 10: | Train Loss: 10.65908
Epoch 11: | Train Loss: 10.63203
Epoch 12: | Train Loss: 10.60279
Epoch 13: | Train Loss: 10.55002
Epoch 14: | Train Loss: 10.53477
Epoch 15: | Train Loss: 10.49371
Epoch 16: | Train Loss: 10.48036
Epoch 17: | Train Loss: 10.47342
Epoch 18: | Train Loss: 10.45329
Epoch 19: | Train Loss: 10.44321
Epoch 20: | Train Loss: 10.42964
Epoch 21: | Train Loss: 10.41264
Epoch 22: | Train Loss: 10.40726
Epoch 23: | Train Loss: 10.40710
Epoch 24: | Train Loss: 10.39566
Epoch 25: | Train Loss: 10.39069
Epoch 26: | Train Loss: 10.38748
Epoch 27: | Train Loss: 10.36426
Epoch 28: | Train Loss: 10.36624
Epoch 29: | Train Loss: 10.35305
Epoch 30: | Train Loss: 10.35634
Ep

In [146]:
y_pred_list = []
with torch.no_grad():
    model.eval()
    for X_batch, _ in test_dataloader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_pred_list.append(y_test_pred.cpu().numpy())
        
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [147]:
y_pred_list = np.array(y_pred_list[0])
# y
y_pred_list.shape

(64, 2)